In [26]:
from data_pipeline.environments.cubby_environment import CubbyEnvironment 
from robofin.collision import FrankaSelfCollisionChecker

env = CubbyEnvironment()
selfcc = FrankaSelfCollisionChecker()

In [27]:
obstacles, qs_free, poses_free, qs_collision, poses_collision = env.sample_q_pose(selfcc=selfcc, how_many=50, margin=0)
print("len(qs_free):", len(qs_free))
print("len(qs_collision):", len(qs_collision))

len(qs_free): 50
len(qs_collision): 639


In [28]:
import torch
from robofin.pointcloud.torch import FrankaSampler
fk_sampler = FrankaSampler("cpu", use_cache=True)

qs = qs_collision
q = torch.tensor(qs[5], dtype=torch.float32, requires_grad=False)
robot_points = fk_sampler.sample(q, 2048)
print("robot_points:", robot_points.shape)

robot_points: torch.Size([1, 2048, 3])


In [29]:
from data_pipeline.geometry import construct_mixed_point_cloud
obstacle_points = construct_mixed_point_cloud(obstacles, num_points=4096)

In [30]:
import open3d as o3d

# Create an Open3D PointCloud object for the robot points
robot_pcd = o3d.geometry.PointCloud()
robot_points = robot_points.squeeze().cpu().numpy()  # Convert to numpy array
robot_points = robot_points.reshape(-1, 3)  # Ensure correct shape
robot_pcd.points = o3d.utility.Vector3dVector(robot_points)  # Use precomputed robot_points
robot_pcd.paint_uniform_color([1, 0, 0])  # Red for robot points

# Create an Open3D PointCloud object for the obstacle points
obstacle_pcd = o3d.geometry.PointCloud()
obstacle_pcd.points = o3d.utility.Vector3dVector(obstacle_points[:, :3])  # Use precomputed obstacle_points
obstacle_pcd.paint_uniform_color([0, 1, 0])  # Green for obstacle points

# Combine the two point clouds into a single list
pcd = [robot_pcd, obstacle_pcd]

# Visualize both point clouds together
o3d.visualization.draw_geometries(pcd)
